Test liveimport notebook integration.  The notebook can be tested automatically
or manually.

Code cells declare to the automatic tester what it should expect.

|Declaration|Tester should expect|
|---|-------|
|`#@ reload` *module*| A Markdown output line "Reloaded *module* ..." |
|`#@ error` *ExceptionType*| An error output for *ExceptionType* |
|`#@ missingok` | No "OK" Markdown output |

Unless there is a `#@ missingok` directive, the automatic test fails if any
cell does have an "OK" Markdown output.  `#@ reload` and `@# error`
declarations are exclusive; the automatic tester both requires those reloads
and errors and permits no others.  Furthermore, the order of reloads and errors
in the output must match the order of their respective declarations.

Manual testing procedure:

1.  Clear all cell output
2.  Restart the server
3.  Run all cells

If the output of every code cell includes "OK" at the end except for the
intentional RuntimeException cell in the "Cell Magic" section which should be
terminated by that exception, then all tests passed.  Be aware that some cells
should include tracebacks, but still have "OK" output.

(The notebook is designed to work without #2, but it is probably best for
certainty.)

Still need to manually test for

* Auto syncing after more than a non-zero grace period expires

In [ ]:
import liveimport
from IPython.display import Markdown

from common import *

describe_environment()

# globals() access means this can't be defined in setup
def is_registered(modulename:str, name:str|None=None, asname:str|None=None):
    return liveimport._is_registered(globals(),modulename,name,asname)

def ok():
    display(Markdown("OK"))

ok()

### Explicit Registration

#### With grace=1.0

In [ ]:
#
# Register using the three different import statement forms.
#
# [See coreapi.test_three_forms().]
#

liveimport.register(globals(),"""
import mod1
from mod2 import mod2_public1, mod2_public2 as mod2_public2_alias
from mod3 import *
from mod4 import *
""", clear=True)

assert is_registered('mod1')
assert is_registered('mod2','mod2_public1')
assert is_registered('mod2','mod2_public2','mod2_public2_alias')
assert is_registered('mod3','*')
assert is_registered('mod4','*')
assert not is_registered('mod1','*')
assert not is_registered('mod1','mod1_public1')
assert not is_registered('mod2','*')
assert not is_registered('mod2','mod2_public2')
assert not is_registered('mod2','mod2_public3')
assert not is_registered('mod3','mod3_public1')
assert not is_registered('mod4','mod4_public1')

ok()

In [ ]:
#
# Prepare for next cell
#

mod1_tag = get_tag("mod1")
mod2_tag = get_tag("mod2")
mod3_tag = get_tag("mod3")
mod4_tag = get_tag("mod4")

touch_module("mod1")
touch_module("mod2")
touch_module("mod3")
touch_module("mod4")

ok()

In [ ]:
#
# Updated modules should NOT trigger auto sync because the grace period
# has not expired.
#

expect_tag("mod1",mod1_tag)
expect_tag("mod2",mod2_tag)
expect_tag("mod3",mod3_tag)
expect_tag("mod4",mod4_tag)

ok()

#### With grace=0.0

Setting grace to zero enables us to trigger autosync when running straight
through.

In [ ]:
#
# Prepare for the next cell.  
#

liveimport.auto_sync(grace=0.0)

ok()

In [ ]:
#
# All registered modules should have automatically reloaded.
#

expect_tag("mod1",next_tag(mod1_tag))
expect_tag("mod2", next_tag(mod2_tag))
expect_tag("mod3",next_tag(mod3_tag))
expect_tag("mod4",next_tag(mod4_tag))

ok()

#@ reload mod1
#@ reload mod2
#@ reload mod3
#@ reload mod4

In [ ]:
#
# Prepare for next cell
#

mod1_tag = get_tag("mod1")
mod2_tag = get_tag("mod2")
mod3_tag = get_tag("mod3")
mod4_tag = get_tag("mod4")

touch_module("mod2")

ok()

In [ ]:
#
# Only mod2 should have reloaded.
#

expect_tag("mod1",mod1_tag)
expect_tag("mod2",next_tag(mod2_tag))
expect_tag("mod3",mod3_tag)
expect_tag("mod4",mod4_tag)
ok()

#@ reload mod2

#### Errors in underlying modules

In [ ]:
#
# Make a registered module syntactically invalid.
#

assert is_registered("mod1")

modify_module("mod1",postscript="not valid python")

ok()

In [ ]:
#
# There should be a syntax error here when liveimport attempts to automatically
# sync the module.
#

ok()

#@ error SyntaxError

In [ ]:
#
# There should again be a syntax error here because it hasn't been fixed yet.
# We do fix it now.
#

restore_module("mod1")
ok()

#@ error SyntaxError

In [ ]:
#
# Module mod1 should have automatically synced without an error.  However, we
# update it to raise a RuntimeError exception.
#

modify_module("mod1",postscript="raise RuntimeError('as expected')")

ok()

#@ reload mod1

In [ ]:
#
# There should be a runtime error here when liveimport attempts to
# automatically sync the module.
#

ok()

#@ error RuntimeError

In [ ]:
#
# There should again be a runtime error here because it hasn't been fixed yet.
# We do fix it now.
#

restore_module("mod1")

ok()

#@ error RuntimeError

In [ ]:
#
# Module unreliable should automatically synced without an error.  
#

ok()

#@ reload mod1

### Cell magic

#### Normal magic

In [ ]:
#
# Reset, except leave grace at zero.
#

liveimport.register(globals(),"",clear=True)
liveimport.auto_sync(enabled=True,grace=0.0)
liveimport.hidden_cell_magic(enabled=False)
body_ran = False
ok()

In [ ]:
%%liveimport
import mod1
from mod2 import mod2_public1, mod2_public2 as mod2_public2_alias
from mod3 import *
from mod4 import *
body_ran = True
ok()

In [ ]:
#
# Verify registrations above, and also that the code executed.
#

assert is_registered('mod1')
assert is_registered('mod2','mod2_public1')
assert is_registered('mod2','mod2_public2','mod2_public2_alias')
assert is_registered('mod3','*')
assert is_registered('mod4','*')
assert not is_registered('mod1','*')
assert not is_registered('mod1','mod1_public1')
assert not is_registered('mod2','*')
assert not is_registered('mod2','mod2_public2')
assert not is_registered('mod2','mod2_public3')
assert not is_registered('mod3','mod3_public1')
assert not is_registered('mod4','mod4_public1')
assert body_ran

ok()

In [ ]:
#
# Prepare for next cell
#

mod1_tag = get_tag("mod1")
mod2_tag = get_tag("mod2")
mod3_tag = get_tag("mod3")
mod4_tag = get_tag("mod4")

touch_module("mod1")
touch_module("mod2")
touch_module("mod3")
touch_module("mod4")

ok()

In [ ]:
#
# All registered modules should have automatically reloaded.
#

expect_tag("mod1",next_tag(mod1_tag))
expect_tag("mod2",next_tag(mod2_tag))
expect_tag("mod3",next_tag(mod3_tag))
expect_tag("mod4",next_tag(mod4_tag))

ok()

#@ reload mod1
#@ reload mod2
#@ reload mod3
#@ reload mod4

In [ ]:
%%liveimport --clear
import mod1
ok()

In [ ]:
#
# Verify --clear option of %%livemagic
#

assert is_registered('mod1')
assert not is_registered('mod2')
assert not is_registered('mod3')
assert not is_registered('mod4')
ok()

In [ ]:
%%liveimport
from mod2 import mod2_public1
raise RuntimeError("Intentional exception")

#@ error RuntimeError
#@ missingok

In [ ]:
#
# Verify that an exception in a %%liveimport cell prevents any registration
#

assert not is_registered("mod2")
ok()

#### Hidden magic

In [ ]:
#_%%liveimport
# pyright: reportMissingImports=false
from mod3 import *
ok()

In [ ]:
#
# Verify hidden cell magic not enabled.
#

assert not is_registered('mod3')
ok()

In [ ]:
#
# Enable hidden cell magic
#

liveimport.hidden_cell_magic(enabled=True)
body_ran = False
ok()

In [ ]:
#_%%liveimport
# pyright: reportMissingImports=false
from mod3 import *
body_ran = True
ok()

In [ ]:
#
# Verify hidden cell magic is enabled.
#
assert body_ran
assert is_registered('mod3')
ok()

### Imports from Packages

Just in case there is something different about how notebook namespaces are
managed.

In [ ]:
#
# Registration of modules in packages.
#

liveimport.register(globals(),"""
import pkg.smod1
from pkg.smod2 import smod2_public1
from pkg.smod3 import *
""", clear=True)

assert is_registered("pkg.smod1")
assert is_registered("pkg.smod2","smod2_public1")
assert is_registered("pkg.smod3","*")

liveimport.hidden_cell_magic(enabled=True)

ok()

In [ ]:
#
# Verify syncing of updated modules in packages
#

smod1_tag = get_tag("pkg.smod1")
smod2_tag = get_tag("pkg.smod2")
smod3_tag = get_tag("pkg.smod3")

touch_module("pkg.smod1")
touch_module("pkg.smod2")
touch_module("pkg.smod3")

liveimport.sync()

expect_tag("pkg.smod1",next_tag(smod1_tag))
expect_tag("pkg.smod2",next_tag(smod2_tag))
expect_tag("pkg.smod3",next_tag(smod3_tag))

ok()

In [ ]:
#
# Prepare to verify cell magic
#

liveimport.register(globals(),"",clear=True)
assert not is_registered("pkg.smod1")
assert not is_registered("pkg.smod2")
ok()

In [ ]:
%%liveimport --clear
import pkg.smod1
ok()

In [ ]:
#
# Verify normal cell magic for modules in packages
#

assert is_registered("pkg.smod1")
ok()

In [ ]:
#_%%liveimport --clear
# pyright: reportMissingImports=false
from pkg.smod2 import smod2_public1
ok()

In [ ]:
#
# Verify hidden cell magic for modules in packages
#

assert is_registered("pkg.smod2")
ok()

### Auto Syncing Options.

#### Auto Sync Disabled

We have already verified automatic syncing above.  Now verify automatic syncing
can be disabled and the explicit syncing works.

In [ ]:
#
# Disable auto syncing and prepare to verifying that it is off.
#

liveimport.auto_sync(enabled=False)

liveimport.register(globals(), "import mod1", clear=True)
mod1_tag = get_tag("mod1")
touch_module("mod1")

ok()

In [ ]:
#
# There should have been no reloading since autosync is disabled.  But an
# explicit sync should reload.
#

expect_tag("mod1",mod1_tag)

liveimport.sync()

expect_tag("mod1",next_tag(mod1_tag))

ok()

liveimport.auto_sync(enabled=True)

#### No Auto Sync Reporting

In [ ]:
#
# Disable reporting and touch mod1.
#

liveimport.auto_sync(report=False)

liveimport.register(globals(), "import mod1", clear=True)
mod1_tag = get_tag("mod1")
touch_module("mod1")

ok()

In [ ]:
#
# Should have synced, but there should be no "Reloaded ..." message.
#

expect_tag("mod1",next_tag(mod1_tag))

liveimport.auto_sync(report=True)

ok()

#@ reload mod1